In [ ]:
import pandas as pd
import glob
from scraped import ScrapedFlight
from pdflight import flights_to_dataframes, shift_datetime, in_task
from joblib import Parallel, delayed
import numpy as np
from scipy import stats

%matplotlib inline

In [ ]:
flights = Parallel(n_jobs=-1)(
    delayed(ScrapedFlight.create_from_file)(i) for i in glob.glob("igc_files/7??_*.igc")
)

In [ ]:
(md, fl, th) = flights_to_dataframes(
    (flight for flight in flights if flight.finish)
)

In [ ]:
display(fl.info())
display(fl.index.dtypes)

In [ ]:
th = th.loc[in_task(th, md)]
th = th[th.dt.total_seconds() >= 120]
th

In [ ]:
# fl.loc[shift_datetime(th.index[0], pd.Timedelta(seconds=40))].alt

In [ ]:
df = pd.DataFrame(index=th.index)
for i in th.index:
    df.loc[i, "v"] = (
        fl.loc[shift_datetime(i, th.loc[i])].alt
        - fl.loc[shift_datetime(i, pd.Timedelta(seconds=40))].alt
    ) / (th.loc[i].total_seconds() - 40)
    df.loc[i, "n"] = (
        fl.loc[shift_datetime(i, pd.Timedelta(seconds=40))].alt - fl.loc[i].alt
    ) / 40
    df.loc[i, "x"] = (
        fl.loc[shift_datetime(i, th.loc[i] - pd.Timedelta(seconds=60))].alt
        - fl.loc[shift_datetime(i, th.loc[i] - pd.Timedelta(seconds=20))].alt
    ) / -40
df["dn"] = df.n / df.v
df["dx"] = df.x / df.v

In [ ]:
dn = df[(df.v >= 1) & (df.dn < 1.5)].groupby(["code", "id", "date"]).dn.mean()
dn

In [ ]:
cc = md.join(dn, how="inner")
cc.plot(x="dn", y="points", kind="scatter")
cc.sort_values("points")

In [ ]:
stats.linregress(np.log(cc.dn), cc.points)